In [16]:
import subprocess
import sys
import math
import pickle
import numpy as np
from BioUtils import *
import os
# import gRNAScores.Rule_Set_2_scoring_v1.analysis.model_comparison as RS2; # Import Doench et al. (2016) old on-target scoring module
from cfd_score_calculator import * # Import Doench et al. (2016) off-target scoring module
#import gRNAScores.azimuth.model_comparison as azimuth # Import Doench et al. (2016) new on-target scoring module
import warnings
warnings.filterwarnings(action='ignore',category=UserWarning) # sklearn complains about loading pickled files from a different version, but they're okay


'''
Receives on-target gRNA string (20-mer gRNA, 3-mer PAM) and string specifying
scoring method to be used ("hsu" for Hsu et al., 2013; "cfd" for Cutting
Frequency Determination, Doench et al., 2016). Calls C++ binary executable to do
the actual scoring. Executable location in binExecPath. Executable depends on
text file with off-target gRNA database, found in gListFilePath. Returns array
containing total score (0-100 scale), score of highest-scoring individual hit
(0-100 scale), number of hits scoring higher than threshold (threshold specified
in C++ methods), and Boolean value specifying if the on-target gRNA was found in
the off-target database or not.
'''
def offTargetScore(gRNA, method, enzyme, pamSeq, pamType, maxNumMisMatches=4, gListFilePath="", binExecPath="offTargetScoringBin"):
    platform = "Linux" # default to linux platform
    if sys.platform == "darwin": # if running on mac yosemite,
        platform = "OSX" # set new osx to access alternate binary file

    if gListFilePath == "": # if no argument was passed to gListFilePath
        if enzyme == "Cas9": # if enzyme is Cas9
            gListFilePath = "gListCas9.txt" # use this list
        elif enzyme == "Cas12": # if enzyme is Cas12
            gListFilePath = "gRNAScores/OffTarget_Scoring/gListCpf1.txt" # use cas12 list


    args = (f"{os.getcwd()}"+"/offTargetScoringBinLinux", gRNA, gListFilePath, method, pamSeq, pamType, str(maxNumMisMatches), "no") # stores command to be passed to console. "no" specifies output format as just scores.
    popen = subprocess.Popen(args, stdout=subprocess.PIPE) # passes command to console
    popen.wait(); # waits?
    output = popen.stdout.read() # get console output
    scores = [float(x) for x in output.decode("utf-8").replace('\n','').split(',')] # parse outputs as floats
    scores[2] = int(scores[2]) # parse third output as integer
    scores[3] = scores[3] == 1 # and fourth as Boolean
    return scores

In [18]:
print(offTargetScore("GTTTCTTGGAGAAGGACGAA", "hsu", "Cas9", "NGG", "NGG", 4))

[99.557857, 0.185721, 4, False]


In [15]:
import os

# Find the directory we executed the script from:
execution_dir = os.getcwd()

# Find the directory in which the current script resides:
#file_dir = os.path.dirname(os.path.realpath(__file__))

# Print results
print(f"Execution dir: {execution_dir}")
#print(f"File dir: {file_dir}")


Execution dir: /home/fertroll10/Documents/gRNA_lib/genetargeter/gRNAScores
